<a href="https://colab.research.google.com/github/Hsuan7/homework/blob/main/%E9%87%91%E8%9E%8D%E5%A4%A7%E6%95%B8%E6%93%9Aw10%E4%BD%9C%E6%A5%AD_%E9%87%91%E8%9E%8D%E6%96%87%E6%9C%AC%E6%83%85%E7%B7%92%E8%BE%A8%E8%AD%98Hugging_Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 使用 Hugging Face 框架建立 BERT 模型，針對金融領域文本進行情緒分析訓練。
可使用以下提供的資料集，或選擇其他適合的金融文本資料集。
資料集網址：https://huggingface.co/datasets/takala/financial_phrasebank

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset

# 載入 Financial PhraseBank 資料集
'''
參數2：
sentences_50agree; Number of instances with >=50% annotator agreement: 4846
sentences_66agree: Number of instances with >=66% annotator agreement: 4217
sentences_75agree: Number of instances with >=75% annotator agreement: 3453
sentences_allagree: Number of instances with 100% annotator agreement: 2264

'''

dataset = load_dataset(
    "financial_phrasebank", # 資料集名稱，https://huggingface.co/datasets/takala/financial_phrasebank
    "sentences_allagree", # 指定資料集的子集版本。此處 "sentences_allagree" 表示僅包含所有標註者對情緒標籤達成一致的句子。
    trust_remote_code=True # # 允許執行遠端程式碼。True代表讀取資料集自動載入遠端程式碼。
)

'''
class_label:
names:
'0': negative
'1': neutral
'2': positive
'''

# 查看數據集的一些樣本
print(dataset["train"][0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.88k [00:00<?, ?B/s]

financial_phrasebank.py:   0%|          | 0.00/6.04k [00:00<?, ?B/s]

FinancialPhraseBank-v1.0.zip:   0%|          | 0.00/682k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2264 [00:00<?, ? examples/s]

{'sentence': 'According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .', 'label': 1}


In [3]:
# 查看資料集的基本結構和數據集大小
print(dataset)

# 查看資料集的欄位名稱與類型
print(dataset["train"].column_names)

# 查看訓練集中前幾筆數據
print(dataset["train"][:5])

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 2264
    })
})
['sentence', 'label']
{'sentence': ['According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .', "For the last quarter of 2010 , Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier , while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m .", 'In the third quarter of 2010 , net sales increased by 5.2 % to EUR 205.5 mn , and operating profit by 34.9 % to EUR 23.5 mn .', 'Operating profit rose to EUR 13.1 mn from EUR 8.7 mn in the corresponding period in 2007 representing 7.7 % of net sales .', 'Operating profit totalled EUR 21.1 mn , up from EUR 18.6 mn in 2007 , representing 9.7 % of net sales .'], 'label': [1, 2, 2, 2, 2]}


In [4]:
# 查看資料集中有哪些 Keys
dataset.keys()

dict_keys(['train'])

In [5]:
from collections import Counter # 計算列表中每個元素的出現次數

# 計算標籤出現次數
label_counts = Counter(dataset["train"]["label"])
print(label_counts)

Counter({1: 1391, 2: 570, 0: 303})


In [6]:
import pandas as pd  # 結構化資料處理套件

# 轉換為 DataFrame
df = pd.DataFrame(dataset["train"])
print(df.head())

                                            sentence  label
0  According to Gran , the company has no plans t...      1
1  For the last quarter of 2010 , Componenta 's n...      2
2  In the third quarter of 2010 , net sales incre...      2
3  Operating profit rose to EUR 13.1 mn from EUR ...      2
4  Operating profit totalled EUR 21.1 mn , up fro...      2


In [7]:
# 檢查空值
print(df.isnull().sum())

# 刪除重複值
df.drop_duplicates(subset=["sentence"], inplace=True)
print(df.shape)

sentence    0
label       0
dtype: int64
(2259, 2)


In [8]:
# 將資料集分為訓練集與測試集
train_test_split = dataset["train"].train_test_split(test_size=0.2)

# 確認分割後的資料集大小
print(train_test_split)

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 1811
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 453
    })
})


In [9]:
from transformers import AutoTokenizer  # AutoTokenizer 可以根據指定的模型名稱，自動選擇和載入與該模型相匹配的分詞器

# 使用BERT Tokenizer 進行Tokenization(分詞)
# 如果是中文文本則使用 bert-base-chinese 模型
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# 定義 Tokenize 函數，用於將句子轉換為 Token
def tokenize_function(example):
    # 使用 tokenizer 將每個句子進行分詞
    # 設定 padding="max_length" 確保所有句子補齊至固定長度
    # 設定 truncation=True 以截斷超過最大長度的句子
    return tokenizer(example["sentence"], padding="max_length", truncation=True)

# 對資料集進行 Tokenize
tokenized_datasets = train_test_split.map(tokenize_function, batched=True)

'''
sentence：原始文字內容，這裡是 "Financial details were n't disclosed ."。此句子會被 Tokenizer 切分並轉換成數字。

label：句子的標籤

input_ids：是一個 input_ids 的列表，包含將句子分詞（Tokenization）後對應的 ID 列表。
每個數字（例如 101）對應一個詞元（token），這些數字是根據模型的詞彙表（vocabulary）進行的編碼。
例如，101 通常是特殊的 [CLS] token，用於標記句子開始位置。
'''

# 查看 Tokenized 資料
tokenized_datasets["train"][:1]

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/1811 [00:00<?, ? examples/s]

Map:   0%|          | 0/453 [00:00<?, ? examples/s]

{'sentence': ['The Helsinki-based company , which also owns the Salomon , Atomic and Suunto brands , said net profit rose 15 percent in the three months through Dec. 31 to (  x20ac ) 47 million ( $ 61US million ) , from (  x20ac ) 40.8 million a year earlier .'],
 'label': [2],
 'input_ids': [[101,
   1996,
   12331,
   1011,
   2241,
   2194,
   1010,
   2029,
   2036,
   8617,
   1996,
   16183,
   19506,
   2078,
   1010,
   9593,
   1998,
   10514,
   16671,
   2080,
   9639,
   1010,
   2056,
   5658,
   5618,
   3123,
   2321,
   3867,
   1999,
   1996,
   2093,
   2706,
   2083,
   11703,
   1012,
   2861,
   2000,
   1006,
   1060,
   11387,
   6305,
   1007,
   4700,
   2454,
   1006,
   1002,
   6079,
   2271,
   2454,
   1007,
   1010,
   2013,
   1006,
   1060,
   11387,
   6305,
   1007,
   2871,
   1012,
   1022,
   2454,
   1037,
   2095,
   3041,
   1012,
   102,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,


In [10]:
# 確認 tokenized_datasets 是否包含需要的欄位
print(tokenized_datasets["train"].features)

{'sentence': Value(dtype='string', id=None), 'label': ClassLabel(names=['negative', 'neutral', 'positive'], id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}


In [11]:
import torch  # 深度學習框架
# 如果有GPU就用GPU，沒有GPU用CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [12]:
from transformers import AutoModelForSequenceClassification  # 序列分類模型

# 載入模型
# 如果是中文文本則使用 bert-base-chinese 模型
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from transformers import TrainingArguments  # 包裝訓練參數

training_args = TrainingArguments(
    output_dir="./results",          # 訓練結果的儲存位置，包括模型檔案和預測結果。
    evaluation_strategy="epoch",     # 設置評估策略為每個訓練輪次（epoch）結束後進行一次評估。
    learning_rate=2e-5,              # 設置學習率，這裡使用 2e-5，通常較小的學習率更適合微調預訓練模型。
    per_device_train_batch_size=4,   # 設定每個裝置（如每張 GPU）的訓練批次大小為 4。
    per_device_eval_batch_size=4,    # 設定每個裝置的評估批次大小為 4。
    num_train_epochs=3,              # 訓練輪次設定為 3，模型將完整遍歷訓練集三次。
    weight_decay=0.01,               # 設置權重衰減（L2正則化）參數，這裡為 0.01，用於防止過擬合。
    logging_dir="./logs",            # 日誌存放目錄，用於儲存 TensorBoard 或其他日誌。
    logging_steps=10,                # 設置每 10 個步驟記錄一次訓練信息，便於監控訓練過程。
    report_to="none"                 # 不將訓練日誌發送到任何平臺（如 TensorBoard），可以改為 "tensorboard" 以啟用。
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids  # 取得真實值的id
    preds = np.argmax(pred.predictions, axis=1)  # 預測值找到最高機率的索引
    accuracy = accuracy_score(labels, preds)  # 計算準確率
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")  # 計算其他指標
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [17]:
import numpy as np


In [18]:

from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

# 開始訓練
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.004800,0.144595,0.975717,0.976143,0.975717,0.975809
2,0.001600,0.162959,0.973510,0.974080,0.973510,0.973531
3,0.000200,0.142276,0.975717,0.975912,0.975717,0.975788


TrainOutput(global_step=1359, training_loss=0.05238315368683171, metrics={'train_runtime': 640.3617, 'train_samples_per_second': 8.484, 'train_steps_per_second': 2.122, 'total_flos': 1429495198516224.0, 'train_loss': 0.05238315368683171, 'epoch': 3.0})

In [19]:
# 評估模型在測試集的表現
eval_result = trainer.evaluate()
print(eval_result)

{'eval_loss': 0.1422758251428604, 'eval_accuracy': 0.9757174392935982, 'eval_precision': 0.975911946772874, 'eval_recall': 0.9757174392935982, 'eval_f1': 0.9757879343872032, 'eval_runtime': 15.0054, 'eval_samples_per_second': 30.189, 'eval_steps_per_second': 7.597, 'epoch': 3.0}


In [20]:
# 測試句子
test_texts = [
    "The company's profit has increased significantly this quarter.", # 「本季公司獲利大幅成長」
    "The increase in costs negatively affected the revenue.", # 「成本的增加對收入產生了負面影響」
    "The company's performance remained stable." # 「公司業績保持穩定」
]

test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors="pt").to(device)
# 推論
outputs = model(**test_encodings)

# 取得預測結果
preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()  # 將結果轉回 CPU 以便處理

# 將數字標籤轉換為文字標籤
label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}
predicted_labels = [label_map[pred] for pred in preds]
print(predicted_labels)

['Positive', 'Negative', 'Positive']
